In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter

mylist = []

for chunk in  pd.read_csv('xaa.txt',sep=',',header=0, error_bad_lines= False, low_memory=False,chunksize=1000000):
    mylist.append(chunk)

hf = pd.concat(mylist, axis= 0)
del mylist
df=hf
column=['A_NUMBER','B_NUMBER','TIME','DURATION','CELL1','CELL2']
df.columns=column


df_cdr = df


df_cdr.A_NUMBER=pd.to_numeric(df_cdr.A_NUMBER, errors='coerce').fillna(0).map(int)
df_cdr.B_NUMBER=pd.to_numeric(df_cdr.B_NUMBER, errors='coerce').fillna(0).map(int)
df_cdr.DURATION = pd.to_numeric(df_cdr.DURATION,errors='coerce').fillna(0).map(int)

df_cdr['COUNT']=df_cdr.groupby('CELL2')['CELL2'].transform('count')
df_cdr['CALL_START_TIME'] =  pd.to_timedelta(df_cdr['TIME'], errors ='coerce')
df_cdr['CALL_DURATION'] =  pd.to_timedelta(df_cdr['DURATION'], unit = 'S')
df_cdr['CALL_END_TIME'] = df_cdr['CALL_START_TIME']+df_cdr['CALL_DURATION']

df_cdr[['A_NUMBER','B_NUMBER']] = np.sort(df_cdr[['A_NUMBER','B_NUMBER']], axis=1)
df_cdr['DUPLICATES']= df_cdr.duplicated(subset = ['A_NUMBER','B_NUMBER'], keep = False)
gf = df_cdr[df_cdr['DUPLICATES']]
df_cdr = gf

df_cdr.sort_values(by=['A_NUMBER','B_NUMBER', 'CALL_START_TIME'], inplace=True)
df_cdr.CALL_START_TIME = df_cdr.CALL_START_TIME.shift(-1)
df_cdr['CALL_TIME_DIFFERENCE']=df_cdr['CALL_START_TIME']-df_cdr['CALL_END_TIME']
time_compare = pd.to_timedelta(30, unit = 'S')
time_zero = pd.to_timedelta(0, unit = 'S')
out=df_cdr[((df_cdr['CALL_TIME_DIFFERENCE'] < time_compare )&(df_cdr['CALL_TIME_DIFFERENCE'] > time_zero ))]

out['DROPS']=out.groupby('CELL2')['CELL2'].transform('count')
out['PERCENTAGE']=out['DROPS']/out['COUNT']*100

filtered = out[['CELL2','COUNT','DROPS','PERCENTAGE']]

filtered1 = filtered.drop_duplicates('CELL2')
x=filtered1['COUNT'].mean()
a=filtered1['DROPS'].mean()
filtered1['ATTENTION_SCORE']= (0.4*filtered1['COUNT']/x+0.6*filtered1['DROPS']/a)*filtered1['PERCENTAGE']
filtered1.to_csv('results_filtered.csv', index =False)
out.to_csv('results.csv',index = False)